In [3]:
import pandas as pd
train_df = pd.read_csv("../sales_train.csv.gz")
test_df  = pd.read_csv("../test.csv.gz")
categories = pd.read_csv('../item_categories.csv')
items = pd.read_csv('../items.csv')

In [4]:
%store -r __prepare_data

In [5]:
__prepare_data

In [6]:
%store -r __add_categories

In [7]:
__add_categories

In [9]:
all_data = all_data[all_data.date_block_num > 1] # no lags before time 1
all_data.fillna(0, inplace=True)
print(all_data.isnull().values.any())

False


In [11]:
from sklearn.model_selection import train_test_split

target = all_data.target.values
target[target>20] = 20

predictors = ["target_lag_1","item_target_enc_lag_1","shop_target_enc_lag_1",
                             "item_target_enc_ave",  "shop_target_enc_ave",
             "item_cat_target_enc_lag_1", "shop_item_cat_target_enc_lag_1",
              "shop_super_cat_target_enc_lag_1","super_cat_target_enc_lag_1"
             ]

X_train, X_test, y_train, y_test = \
    train_test_split(all_data[predictors], target, test_size=0.2, random_state=123)

In [12]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

model2 = RandomForestRegressor(n_estimators=64, max_features=3, n_jobs=6) #len(predictors)/2
model2.fit(X_train, y_train)
ypred = model2.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, ypred))
print("RMSE: %f" % (rmse))

RMSE: 0.729407


In [13]:
from sklearn.ensemble import RandomForestRegressor
full_model = RandomForestRegressor(n_estimators=64, max_features=3, n_jobs=6) #len(predictors)/2
full_model.fit(all_data[predictors], target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=3, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=64, n_jobs=6,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [1]:
%store -r __prepare_submission

In [15]:
__prepare_submission

In [27]:
aliases = \
{"target_lag_1":          "target",
 "item_target_enc_lag_1": "item_target_enc",
 "shop_target_enc_lag_1": "shop_target_enc",
 "item_target_enc_ave":   "item_target_enc_ave",
 "shop_target_enc_ave":   "shop_target_enc_ave",
 "item_cat_target_enc_lag_1":      "item_cat_target_enc",
 "shop_item_cat_target_enc_lag_1": "shop_item_cat_target_enc",
 "shop_super_cat_target_enc_lag_1":"shop_super_cat_target_enc",
 "super_cat_target_enc_lag_1":     "super_cat_target_enc"
}
all_data_33.fillna(0, inplace=True)
prepare_submission(full_model, predictors, aliases, "more_features.csv")

In [28]:
from tabulate import tabulate
headers = ["name", "score"]
values = sorted(zip(X_test.columns, full_model.feature_importances_), key=lambda x: x[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

name                                 score
target_lag_1                     0.27163
item_target_enc_ave              0.232063
item_target_enc_lag_1            0.15347
shop_item_cat_target_enc_lag_1   0.0850627
item_cat_target_enc_lag_1        0.0606884
shop_super_cat_target_enc_lag_1  0.0574929
super_cat_target_enc_lag_1       0.0529111
shop_target_enc_lag_1            0.0478585
shop_target_enc_ave              0.038824


In [17]:
predictors = ["date_block_num", "shop_id", "item_id"] + \
             ["target_lag_1",
              "item_target_enc_lag_1",
              "shop_target_enc_lag_1",
              "shop_item_cat_target_enc_lag_1",
              "shop_super_cat_target_enc_lag_1",
              "super_cat_target_enc_lag_1"
             ]
#all_data[predictors].to_csv("alt.csv")

In [18]:
all_data[predictors].tail()

,date_block_num,shop_id,item_id,target_lag_1,item_target_enc_lag_1,shop_target_enc_lag_1,shop_item_cat_target_enc_lag_1,shop_super_cat_target_enc_lag_1,super_cat_target_enc_lag_1
10913845,33,59,22162,0.0,0.162791,0.179744,0.109834,0.112623,0.197367
10913846,33,59,22163,0.0,0.465116,0.179744,0.109834,0.112623,0.197367
10913847,33,59,22164,0.0,0.209302,0.179744,0.131965,0.112623,0.197367
10913848,33,59,22166,0.0,0.116279,0.179744,0.000000,0.033943,0.189629
10913849,33,59,22167,0.0,0.488372,0.179744,0.024390,0.033943,0.189629
